In [1]:
val julio = "NASA_Access_Log_Jul95.gz"

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002842.bosonit.local:4041
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1629458267514)
SparkSession available as 'spark'


julio: String = NASA_Access_Log_Jul95.gz


In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
.builder
.appName("Nasa")
.getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6c49d9f0


In [3]:
val nasaLogsJulio = spark.read.option("sep"," ").csv(julio)

nasaLogsJulio: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 6 more fields]


In [4]:
nasaLogsJulio.show(1,false)

+------------+---+---+---------------------+------+-----------------------------+---+----+
|_c0         |_c1|_c2|_c3                  |_c4   |_c5                          |_c6|_c7 |
+------------+---+---+---------------------+------+-----------------------------+---+----+
|199.72.81.55|-  |-  |[01/Jul/1995:00:00:01|-0400]|GET /history/apollo/ HTTP/1.0|200|6245|
+------------+---+---+---------------------+------+-----------------------------+---+----+
only showing top 1 row



In [5]:
import org.apache.spark.sql.functions.regexp_replace

val nasaLogsJulio1 = nasaLogsJulio.withColumn("_c3", regexp_replace($"_c3", "\\[", ""))
val nasaLogsJulio2 = nasaLogsJulio1.withColumn("_c4", regexp_replace($"_c4", "\\]", ""))
val nasaLogsJulio3 = nasaLogsJulio2.withColumn("RequestMethod", regexp_replace($"_c5", "(\\w+).*", "$1"))
val nasaLogsJulio4 = nasaLogsJulio3.withColumn("Resource", regexp_replace($"_c5", "\\S+\\s(\\S+).*", "$1"))
val nasaLogsJulio5 = nasaLogsJulio4.withColumn("Protocol", regexp_replace($"_c5", "\\S+\\s+\\S+\\s(\\S+).*", "$1"))
val nasaLogsJulio6 = nasaLogsJulio5.drop("_c5")

import org.apache.spark.sql.functions.regexp_replace
nasaLogsJulio1: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 6 more fields]
nasaLogsJulio2: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 6 more fields]
nasaLogsJulio3: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 7 more fields]
nasaLogsJulio4: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 8 more fields]
nasaLogsJulio5: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 9 more fields]
nasaLogsJulio6: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 8 more fields]


In [6]:
nasaLogsJulio6.show(1,false)

+------------+---+---+--------------------+-----+---+----+-------------+----------------+--------+
|_c0         |_c1|_c2|_c3                 |_c4  |_c6|_c7 |RequestMethod|Resource        |Protocol|
+------------+---+---+--------------------+-----+---+----+-------------+----------------+--------+
|199.72.81.55|-  |-  |01/Jul/1995:00:00:01|-0400|200|6245|GET          |/history/apollo/|HTTP/1.0|
+------------+---+---+--------------------+-----+---+----+-------------+----------------+--------+
only showing top 1 row



In [7]:
val nasaLogsJulio7 = nasaLogsJulio6.withColumnRenamed("_c0", "Host")
                             .withColumnRenamed("_c1", "UserIdentifier")
                             .withColumnRenamed("_c2", "UserId")
                             .withColumnRenamed("_c4", "TimeZone")
                             .withColumnRenamed("_c6", "HttpStatus")
                             .withColumnRenamed("_c7", "Size")

nasaLogsJulio7: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 8 more fields]


In [8]:
nasaLogsJulio7.show(1,false)

+------------+--------------+------+--------------------+--------+----------+----+-------------+----------------+--------+
|Host        |UserIdentifier|UserId|_c3                 |TimeZone|HttpStatus|Size|RequestMethod|Resource        |Protocol|
+------------+--------------+------+--------------------+--------+----------+----+-------------+----------------+--------+
|199.72.81.55|-             |-     |01/Jul/1995:00:00:01|-0400   |200       |6245|GET          |/history/apollo/|HTTP/1.0|
+------------+--------------+------+--------------------+--------+----------+----+-------------+----------------+--------+
only showing top 1 row



In [9]:
val nasaLogs5 = nasaLogsJulio7.withColumn("DateTime", to_timestamp($"_c3", "dd/MMM/yyyy:HH:mm:ss"))

nasaLogs5: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 9 more fields]


In [10]:
val nasaLogs4 = nasaLogs5.drop("_c3")
nasaLogs4.printSchema()
nasaLogs4.show(1,false)

root
 |-- Host: string (nullable = true)
 |-- UserIdentifier: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- TimeZone: string (nullable = true)
 |-- HttpStatus: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- RequestMethod: string (nullable = true)
 |-- Resource: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)

+------------+--------------+------+--------+----------+----+-------------+----------------+--------+-------------------+
|Host        |UserIdentifier|UserId|TimeZone|HttpStatus|Size|RequestMethod|Resource        |Protocol|DateTime           |
+------------+--------------+------+--------+----------+----+-------------+----------------+--------+-------------------+
|199.72.81.55|-             |-     |-0400   |200       |6245|GET          |/history/apollo/|HTTP/1.0|1995-07-01 00:00:01|
+------------+--------------+------+--------+----------+----+-------------+----------------+--------

nasaLogs4: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 8 more fields]


In [11]:
import org.apache.spark.sql.functions._

val nasaLogs3 = nasaLogs4.withColumn("Date",to_date($"DateTime"))
val nasaLogs2 = nasaLogs3.withColumn("Time",date_format($"DateTime", "HH:mm:ss"))
val nasaLogs1 = nasaLogs2.drop("DateTime")

import org.apache.spark.sql.functions._
nasaLogs3: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 9 more fields]
nasaLogs2: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 10 more fields]
nasaLogs1: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 9 more fields]


In [12]:
val nasaLogs = nasaLogs1.select("Host","UserIdentifier","UserId","Date","Time","TimeZone","RequestMethod","Resource","Protocol",
                                     "HttpStatus","Size")

nasaLogs: org.apache.spark.sql.DataFrame = [Host: string, UserIdentifier: string ... 9 more fields]


In [13]:
nasaLogs.printSchema()
nasaLogs.show(1,false)

root
 |-- Host: string (nullable = true)
 |-- UserIdentifier: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- TimeZone: string (nullable = true)
 |-- RequestMethod: string (nullable = true)
 |-- Resource: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- HttpStatus: string (nullable = true)
 |-- Size: string (nullable = true)

+------------+--------------+------+----------+--------+--------+-------------+----------------+--------+----------+----+
|Host        |UserIdentifier|UserId|Date      |Time    |TimeZone|RequestMethod|Resource        |Protocol|HttpStatus|Size|
+------------+--------------+------+----------+--------+--------+-------------+----------------+--------+----------+----+
|199.72.81.55|-             |-     |1995-07-01|00:00:01|-0400   |GET          |/history/apollo/|HTTP/1.0|200       |6245|
+------------+--------------+------+----------+--------+--------+--------

Consultas a realizar:  
    1) - ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.  
    2) - ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.  
    3) - ¿Y los métodos de petición (verbos) más utilizados?  
    4) - ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?  
    5) - Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros 
         en nuestro log.  
    6) - ¿Qué días la web recibió más tráfico?  
    7) - ¿Cuáles son los hosts son los más frecuentes?  
    8) - ¿A qué horas se produce el mayor número de tráfico en la web?  
    9) - ¿Cuál es el número de errores 404 que ha habido cada día?  

### Respuestas:  
1) - ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.

In [14]:
val uno1 = nasaLogs.groupBy("Protocol").count().limit(1)
val uno = uno1.withColumn("Consulta", lit("Protocolo mas usado"))

uno1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: bigint]
uno: org.apache.spark.sql.DataFrame = [Protocol: string, count: bigint ... 1 more field]


2) - ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.

In [15]:
val dos1 = nasaLogs.groupBy("HttpStatus").count().sort(col("count").desc).limit(1)
val dos = dos1.withColumn("Consulta", lit("HttpStatus mas comun"))

dos1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [HttpStatus: string, count: bigint]
dos: org.apache.spark.sql.DataFrame = [HttpStatus: string, count: bigint ... 1 more field]


3) - ¿Y los métodos de petición (verbos) más utilizados?

In [16]:
val tres1 = nasaLogs.groupBy("RequestMethod").count().sort(col("count").desc).limit(1)
val tres = tres1.withColumn("Consulta", lit("RequestMethod mas utilizado"))

tres1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [RequestMethod: string, count: bigint]
tres: org.apache.spark.sql.DataFrame = [RequestMethod: string, count: bigint ... 1 more field]


4) - ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?

In [17]:
val cuatro1 = nasaLogs.select(col("Resource"), col("Size")).orderBy(col("Size").desc).limit(1)
val cuatro = cuatro1.withColumn("Consulta", lit("Resource con mas transferencia de bytes"))

cuatro1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Resource: string, Size: string]
cuatro: org.apache.spark.sql.DataFrame = [Resource: string, Size: string ... 1 more field]


5) - Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros 
         en nuestro log.

In [18]:
val cinco1 = nasaLogs.groupBy("Resource").count().sort(col("count").desc).limit(1)
val cinco = cinco1.withColumn("Consulta", lit("Resource con mas registros"))

cinco1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Resource: string, count: bigint]
cinco: org.apache.spark.sql.DataFrame = [Resource: string, count: bigint ... 1 more field]


6) - ¿Qué días la web recibió más tráfico?

In [19]:
val seis1 = nasaLogs.select(date_format($"Date","d").alias("Dia"))
                                .groupBy(col("Dia"))
                                .agg(count("*").alias("Trafico"))
                                .orderBy(col("Trafico").desc)
                                .limit(1)
val seis = seis1.withColumn("Consulta", lit("Dia con mas trafico"))

seis1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Dia: string, Trafico: bigint]
seis: org.apache.spark.sql.DataFrame = [Dia: string, Trafico: bigint ... 1 more field]


7) - ¿Cuáles son los hosts más frecuentes? 

In [20]:
val siete1 = nasaLogs.groupBy("Host").count().sort(col("count").desc).limit(1)
val siete = siete1.withColumn("Consulta", lit("Host mas frecuente"))

siete1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Host: string, count: bigint]
siete: org.apache.spark.sql.DataFrame = [Host: string, count: bigint ... 1 more field]


8) - ¿A qué horas se produce el mayor número de tráfico en la web?

In [21]:
val ocho1 = nasaLogs.select(date_format($"Time","H").alias("Hora"))
                                .groupBy(col("Hora"))
                                .agg(count("*").alias("Trafico"))
                                .orderBy(col("Trafico").desc)
                                .limit(1)
val ocho = ocho1.withColumn("Consulta", lit("Hora con mayor trafico"))

ocho1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: string, Trafico: bigint]
ocho: org.apache.spark.sql.DataFrame = [Hora: string, Trafico: bigint ... 1 more field]


9) - ¿Cuál es el número de errores 404 que ha habido cada día?

In [22]:
val nueve1 = nasaLogs.select(date_format($"Date","d").alias("Dia"),col("HttpStatus"))
                                .where(col("HttpStatus") === 404)
                                .groupBy(col("Dia"))
                                .agg(count("*").alias("Errores"))
                                .orderBy(col("Errores").desc)
                                .limit(1)
val nueve = nueve1.withColumn("Consulta", lit("Dia con mayor numero de error 404"))

nueve1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Dia: string, Errores: bigint]
nueve: org.apache.spark.sql.DataFrame = [Dia: string, Errores: bigint ... 1 more field]


In [23]:
val diez = uno.union(dos)
val once = diez.union(tres)
val doce = once.union(cuatro)
val trece = doce.union(cinco)
val catorce = trece.union(seis)
val quince = catorce.union(siete)
val dieciseis = quince.union(ocho)
val diecisiete = dieciseis.union(nueve)

diez: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: bigint ... 1 more field]
once: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: bigint ... 1 more field]
doce: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: string ... 1 more field]
trece: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: string ... 1 more field]
catorce: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: string ... 1 more field]
quince: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: string ... 1 more field]
dieciseis: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Protocol: string, count: string ...


In [24]:
val dieciocho = diecisiete.withColumnRenamed("Protocol","Respuesta")
val diecinueve = dieciocho.select("Consulta", "Respuesta", "count")

dieciocho: org.apache.spark.sql.DataFrame = [Respuesta: string, count: string ... 1 more field]
diecinueve: org.apache.spark.sql.DataFrame = [Consulta: string, Respuesta: string ... 1 more field]


In [25]:
diecinueve.show(truncate=false)

+---------------------------------------+------------------------------+-------+
|Consulta                               |Respuesta                     |count  |
+---------------------------------------+------------------------------+-------+
|Protocolo mas usado                    |GET /ksc.html                 |159    |
|HttpStatus mas comun                   |200                           |1701534|
|RequestMethod mas utilizado            |GET                           |1887643|
|Resource con mas transferencia de bytes|/images/cdrom-1-95/img0007.jpg|99981  |
|Resource con mas registros             |/images/NASA-logosmall.gif    |111388 |
|Dia con mas trafico                    |13                            |134203 |
|Host mas frecuente                     |piweba3y.prodigy.com          |17572  |
|Hora con mayor trafico                 |14                            |122479 |
|Dia con mayor numero de error 404      |6                             |640    |
+---------------------------